<a href="https://colab.research.google.com/github/Tuttotorna/lon-mirror/blob/main/artifacts/demos/DELTA_policy_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DELTA-policy demo (stand-alone)

from dataclasses import dataclass
from enum import Enum
from typing import List, Tuple, Dict
import re

class Boundary(str, Enum):
    OPEN = "open"
    NEAR_BOUNDARY = "near_boundary"
    SATURATED = "saturated"

class Loss(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    CRITICAL = "critical"

class Compressibility(str, Enum):
    YES = "yes"
    PARTIAL = "partial"
    NO = "no"

class Action(str, Enum):
    ALLOW = "ALLOW"
    BOUNDARY_ONLY = "BOUNDARY_ONLY"
    REFUSE = "REFUSE"

@dataclass(frozen=True)
class DeltaDecision:
    B: Boundary
    L: Loss
    C: Compressibility
    action: Action
    reasons: List[str]
    scores: Dict[str, float]

_NUM_RE = re.compile(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?")

def _extract_numbers(text: str):
    return _NUM_RE.findall(text)

def _tail_sentences(text: str, n: int = 2) -> str:
    parts = re.split(r"(?<=[.!?])\s+", text.strip())
    return " ".join(parts[-n:]) if parts else text.strip()

def conclusion_signature(text: str) -> Tuple[str, int, Tuple[str, ...]]:
    tail = _tail_sentences(text, 2).lower()
    neg = len(re.findall(r"\b(no|not|never|cannot|can't|impossible)\b", tail))
    nums = tuple(_extract_numbers(tail))
    return (tail, neg, nums)

def T_score(instances: List[str]) -> float:
    sigs = [conclusion_signature(x) for x in instances if x and x.strip()]
    if len(sigs) < 2:
        return 0.0
    tails = [s[0] for s in sigs]
    negs = [s[1] for s in sigs]
    nums = [s[2] for s in sigs]
    tail_div = 1.0 if len(set(tails)) > 1 else 0.0
    neg_div = 1.0 if len(set(negs)) > 1 else 0.0
    num_div = 1.0 if len(set(nums)) > 1 else 0.0
    return min(1.0, 0.5*tail_div + 0.25*neg_div + 0.25*num_div)

def I_score(text: str) -> float:
    t = (text or "").lower()
    if not t.strip():
        return 0.0
    abs_q = bool(re.search(r"\b(always|never|must|cannot|impossible)\b", t))
    exc_q = bool(re.search(r"\b(sometimes|may|might|can|often|in some cases)\b", t))
    qualifier_conflict = 1.0 if (abs_q and exc_q) else 0.0
    nums = _extract_numbers(t)
    numeric_density = min(1.0, len(nums)/12.0)
    return min(1.0, 0.7*qualifier_conflict + 0.3*numeric_density)

def S_score(prompt: str, text: str) -> float:
    p = (prompt or "")
    a = (text or "")
    if not a.strip():
        return 0.0
    lp = max(1, len(p))
    la = len(a)
    length_ratio = min(1.0, la/(6.0*lp))
    because_chain = len(re.findall(r"\b(because|therefore|thus|hence)\b", a.lower()))
    chain_score = min(1.0, because_chain/8.0)
    return min(1.0, 0.6*length_ratio + 0.4*chain_score)

def delta_policy(prompt, draft, variants, t_hi=0.70, t_mid=0.40, i_hi=0.60, s_hi=0.60):
    instances = [draft] + list(variants or [])
    t = T_score(instances)
    i = I_score(draft)
    s = S_score(prompt, draft)
    reasons=[]
    if t>=t_hi: reasons.append("instability_high")
    elif t>=t_mid: reasons.append("instability_mid")
    if i>=i_hi: reasons.append("contradiction_high")
    if s>=s_hi: reasons.append("saturation_high")
    if i>=i_hi or t>=t_hi:
        return DeltaDecision(Boundary.SATURATED, Loss.CRITICAL, Compressibility.NO, Action.REFUSE, reasons, {"T":t,"I":i,"S":s})
    elif s>=s_hi or t>=t_mid:
        return DeltaDecision(Boundary.NEAR_BOUNDARY, Loss.MEDIUM, Compressibility.PARTIAL, Action.BOUNDARY_ONLY, reasons, {"T":t,"I":i,"S":s})
    else:
        return DeltaDecision(Boundary.OPEN, Loss.LOW, Compressibility.YES, Action.ALLOW, reasons, {"T":t,"I":i,"S":s})

# ---- Demo cases ----
prompt = "Explain whether this problem has a unique solution."
draft = "This problem always has a unique solution because constraints are sufficient. However, in some cases it may not."
variants = [
    "List constraints then conclude succinctly: It has a unique solution.",
    "Answer in two sentences: It may have multiple solutions depending on constraints."
]

decision = delta_policy(prompt, draft, variants)
decision

DeltaDecision(B=<Boundary.SATURATED: 'saturated'>, L=<Loss.CRITICAL: 'critical'>, C=<Compressibility.NO: 'no'>, action=<Action.REFUSE: 'REFUSE'>, reasons=['instability_high', 'contradiction_high'], scores={'T': 0.75, 'I': 0.7, 'S': 0.2696078431372549})